In [1]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
import json
import os, tempfile
from tqdm import tqdm 

In [2]:
dir_path = '../results/'

In [4]:
with open('../data/selected_protein.json', "r") as json_file:
    selected_protein = json.load(json_file)

1431

In [5]:
# load files 
def get_contact(pdb): 
    contact_dict = {}
    with open(dir_path + 'esm2_jac_contact/' + pdb + '_esm2_jac_contact.pkl', 'rb') as f:
        contact_dict['esm2_jac'] = pickle.load(f)

    with open(dir_path + 'msa_contact/' + pdb + '_msa_contact.pkl', 'rb') as f:
        contact_dict['msa'] = pickle.load(f)

    contact_dict['esm2'] = np.load('../results/contacts_' + pdb + '.npy')

    contact_dict['exp_pdb'] = np.load('../results/'+ pdb + '.npy')

    contact_dict['esmfold'] = np.load('../results/contacts' + pdb + '.npy')
    
    return contact_dict 

In [6]:
os.makedirs("hhsuite", exist_ok=True)
os.system(f"curl -fsSL https://github.com/soedinglab/hh-suite/releases/download/v3.3.0/hhsuite-3.3.0-SSE2-Linux.tar.gz | tar xz -C hhsuite/")
#if "hhsuite" not in os.environ['PATH']:
os.environ['PATH'] += ":hhsuite/bin:hhsuite/scripts"

In [7]:
def parse_hhalign_output(filename):
    with open(filename, "r") as f:
        lines = f.readlines()
    start_indices, sequences = [None,None], ["",""]
    for line in lines:
        #print(line.rstrip())
        parts = line.split()
        if line.startswith('Q '):
            if start_indices[0] is None:
                start_indices[0] = int(parts[2]) - 1
            sequences[0] += parts[3]
        if line.startswith('T '):
            if start_indices[1] is None:
                start_indices[1] = int(parts[2]) - 1
            sequences[1] += parts[3]
    return sequences, start_indices

def get_template_idx(query_sequence, target_sequence, glob=True):
    print(query_sequence) 
    print(target_sequence)
    with tempfile.NamedTemporaryFile() as tmp_query, \
    tempfile.NamedTemporaryFile() as tmp_target, \
    tempfile.NamedTemporaryFile() as tmp_alignment:
        tmp_query.write(f">query_sequence\n{query_sequence}\n".encode()); tmp_query.flush()
        tmp_target.write(f">target_sequence\n{target_sequence}\n".encode()); tmp_target.flush()
        os.system(f"hhalign {'-glob' if glob else '-loc'} -hide_cons -i {tmp_query.name} -t {tmp_target.name} -o {tmp_alignment.name}")
        X, start_indices = parse_hhalign_output(tmp_alignment.name)
    #print(f"Q\t{X[0]}\nT\t{X[1]}")
    i, j = start_indices
    idx_i, idx_j = [], []
    for q,t in zip(*X):
        if q != "-" and t != "-":
            idx_i.append(i)
            idx_j.append(j)
        if q != "-": i += 1
        if t != "-": j += 1
    return np.array(idx_i), np.array(idx_j)

In [8]:
with open("../data/full_seq_dict.json", "r") as json_file:
    full_seq = json.load(json_file)
with open("../data/exp_pdb_seq.json", "r") as json_file:
    pdb_seq = json.load(json_file)

In [9]:
def top_L(arr, res_list, exp_contact):
    valid_values_with_indices = [(value, (i, j)) for i, row in enumerate(arr) for j, value in enumerate(row) if i in res_list and j in res_list] # pick only res that exist in PDB 
    filtered_indices = [x for x in valid_values_with_indices if abs(x[1][0] - x[1][1]) >= 24] # pick long range interactions used in ESM2 
    sorted_values_with_indices = sorted(filtered_indices, key=lambda x: x[0], reverse=True)
    top_indices = [index for _, index in sorted_values_with_indices[:arr.shape[0]]]
    
    count_0 = sum(1 for i, j in top_indices if exp_contact[np.where(res_list == i), np.where(res_list == j)] == 0)
    count_1 = sum(1 for i, j in top_indices if exp_contact[np.where(res_list == i), np.where(res_list == j)] == 1)
    
    return count_1/(count_0 + count_1)

In [10]:
def compare_contacts(pdb): 
    contacts_dict = get_contact(pdb)
    _, idx_j = get_template_idx(pdb_seq[pdb], full_seq[pdb]) # get the alignment 
    precision_dict = {}
    for contact_type in contacts_dict:
        if contact_type != 'exp_pdb':
            contact = contacts_dict[contact_type]
            precision_dict[contact_type] = top_L(contact, idx_j, contacts_dict['exp_pdb']) 
    return precision_dict 

In [11]:
info_dict = {}
for pdb in tqdm(selected_protein):
    try:
        info_dict[pdb] = compare_contacts(pdb) 
    except EOFError:
        print(f"Error reading from file associated with {pdb}. Skipping...")
        continue

  0%|          | 0/1431 [00:00<?, ?it/s]- 10:14:51.721 INFO: /tmp/tmpkg0fz95q is in A2M, A3M or FASTA format



VDVIALGEPLIQFNSFNPGPLRFVNYFEKHVAGSELNFCIAVVRNHLSCSLIARVGNDEFGKNIIEYSRAQGIDTSHIKVDNESFTGIYFIQRGYPIPMKSELVYYRKGSAGSRLSPEDINENYVRNSRLVHSTGITLAISDNAKEAVIKAFELAKSRSLDTNIRPKLWSSLEKAKETILSILKKYDIEVLITDPDDTKILLDVTDPDEAYRKYKELGVKVLLYKLGSKGAIAYKDNVKAFKDAYKVPVEDPTGAGDAMAGTFVSLYLQGKDIEYSLAHGIAASTLVITVRGDNELTPTLEDAERFLNEFK
MVDVIALGEPLIQFNSFNPGPLRFVNYFEKHVAGSELNFCIAVVRNHLSCSLIARVGNDEFGKNIIEYSRAQGIDTSHIKVDNESFTGIYFIQRGYPIPMKSELVYYRKGSAGSRLSPEDINENYVRNSRLVHSTGITLAISDNAKEAVIKAFELAKSRSLDTNIRPKLWSSLEKAKETILSILKKYDIEVLITDPDDTKILLDVTDPDEAYRKYKELGVKVLLYKLGSKGAIAYKDNVKAFKDAYKVPVEDPTGAGDAMAGTFVSLYLQGKDIEYSLAHGIAASTLVITVRGDNELTPTLEDAERFLNEFKT


- 10:14:52.062 INFO: Alternative alignment: 0

- 10:14:52.065 INFO: 1 alignments done

- 10:14:52.086 INFO: Realigning 1 HMM-HMM alignments using Maximum Accuracy algorithm



Query         query_sequence
Match_columns 311
No_of_seqs    1 out of 2
Neff          1
Searched_HMMs 1
Date          Thu Nov 23 10:14:52 2023
Command       hhalign -glob -hide_cons -i /tmp/tmpkg0fz95q -t /tmp/tmpvj7ra4sj -o /tmp/tmpuqx6eo4j 

 No Hit                             Prob E-value P-value  Score    SS Cols Query HMM  Template HMM
  1 target_sequence                100.0  7E-217  7E-217 1188.2   0.0  311    1-311     2-312 (313)



  0%|          | 1/1431 [00:02<1:09:58,  2.94s/it]- 10:14:54.667 INFO: /tmp/tmpdjxa4vwx is in A2M, A3M or FASTA format



...

100%|█████████▉| 1430/1431 [1:04:14<00:02,  2.47s/it]- 11:19:05.762 INFO: /tmp/tmpby9_bmfq is in A2M, A3M or FASTA format



YFQGVRPAVIAATGLYTPPDSVSNAELVEAFNTYVANFNAANKARIEAGEIEPLQPSSSEFIEKASGIKSRYVVAKPGIVDPDVMRPIIPERSNDELSILAEMAVTAAEQAIERWGKPRERIGAVLCACSNMQRAYPAMAIEVQNALGLGGFAFDMNVACSSATFGLKTAADFVGGGSVDAVLMVNPEICSGHLNFRDRDSHFIFGDVATAAIVERADDAQGGWSILGTKLKTQFSNNIRNNAGFLNRAWPEGRDKADKLFVQQGRKVFKEVVPLVSEMIIEHAREIGIDPHGLKRMWLHQANINMNEIIGRKVLGRDPTRDENVIILDDYANTSSAGSIIAFHKHQDDMAQGDLGLICSFGAGYSAGTVFVQKR
MGSSHHHHHHSSGRENLYFQGVRPAVIAATGLYTPPDSVSNAELVEAFNTYVANFNAANKARIEAGEIEPLQPSSSEFIEKASGIKSRYVVAKPGIVDPDVMRPIIPERSNDELSILAEMAVTAAEQAIERWGKPRERIGAVLCACSNMQRAYPAMAIEVQNALGLGGFAFDMNVACSSATFGLKTAADFVGGGSVDAVLMVNPEICSGHLNFRDRDSHFIFGDVATAAIVERADDAQGGWSILGTKLKTQFSNNIRNNAGFLNRAWPEGRDKADKLFVQQGRKVFKEVVPLVSEMIIEHAREIGIDPHGLKRMWLHQANINMNEIIGRKVLGRDPTRDENVIILDDYANTSSAGSIIAFHKHQDDMAQGDLGLICSFGAGYSAGTVFVQKR


- 11:19:06.160 INFO: Alternative alignment: 0

- 11:19:06.164 INFO: 1 alignments done

- 11:19:06.185 INFO: Realigning 1 HMM-HMM alignments using Maximum Accuracy algorithm



Query         query_sequence
Match_columns 375
No_of_seqs    1 out of 2
Neff          1
Searched_HMMs 1
Date          Thu Nov 23 11:19:06 2023
Command       hhalign -glob -hide_cons -i /tmp/tmpby9_bmfq -t /tmp/tmpnrmvp0ge -o /tmp/tmph254j9kd 

 No Hit                             Prob E-value P-value  Score    SS Cols Query HMM  Template HMM
  1 target_sequence                100.0  2E-273  2E-273 1497.5   0.0  375    1-375    18-392 (392)



100%|██████████| 1431/1431 [1:04:17<00:00,  2.70s/it]


In [12]:
len(info_dict.keys())

1431

In [13]:
info_dict

{'2V78A': {'esm2_jac': 0.7955271565495208,
  'msa': 0.7635782747603834,
  'esm2': 0.9488817891373802,
  'esmfold': 1.0},
 '3MD9A': {'esm2_jac': 0.9176470588235294,
  'msa': 0.8117647058823529,
  'esm2': 0.9725490196078431,
  'esmfold': 1.0},
 '3LVUA': {'esm2_jac': 0.8449612403100775,
  'msa': 0.627906976744186,
  'esm2': 0.8837209302325582,
  'esmfold': 1.0},
 '3SILA': {'esm2_jac': 0.7519788918205804,
  'msa': 0.5118733509234829,
  'esm2': 0.8680738786279684,
  'esmfold': 1.0},
 '2DWUA': {'esm2_jac': 0.8623188405797102,
  'msa': 0.644927536231884,
  'esm2': 0.9202898550724637,
  'esmfold': 1.0},
 '3KIZA': {'esm2_jac': 0.7715736040609137,
  'msa': 0.583756345177665,
  'esm2': 0.8375634517766497,
  'esmfold': 0.9949238578680203},
 '2HF9A': {'esm2_jac': 0.8761061946902655,
  'msa': 0.504424778761062,
  'esm2': 0.911504424778761,
  'esmfold': 1.0},
 '3C96A': {'esm2_jac': 0.824390243902439,
  'msa': 0.6829268292682927,
  'esm2': 0.8634146341463415,
  'esmfold': 1.0},
 '4JBBA': {'esm2_jac': 

In [14]:
with open(dir_path + 'precision_24res_apart.json', 'w') as file:
    json.dump(info_dict, file)